# Multi Fabric - Commissioning and Interactions

<a href="http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fproject-chip%2Fconnectedhomeip&urlpath=lab%2Ftree%2Fconnectedhomeip%2Fdocs%2Fguides%2Frepl%2FMatter%2520-%2520Multi%2520Fabric%2520Commissioning.ipynb&branch=master">
<img src="https://i.ibb.co/hR3yWsC/launch-playground.png" alt="drawing" width="130"/>
</a>
<br></br>

This walks through creating multiple controllers on multiple fabrics, using those controllers to commission a target onto those fabrics and finally, interacting with them using the interaction model.

## FabricAdmins and Controllers

The `FabricAdmin` class (present in the `chip.FabricAdmin` package) is responsible for adminstering a fabric. It houses the Fabric ID and Index, as well as an RCAC and ICAC that provides the certificate material grounding that fabric.

The `FabricAdmin` can be used to vend `ChipDeviceController` objects that represent a controller instance with a specific identity grounded in the admin's fabric. This controller can then be used to commission and interact with devices.

## Clear Persisted Storage

Let's clear out our persisted storage (if one exists) to start from a clean slate.

In [ ]:
import os, subprocess

if os.path.isfile('/tmp/repl-storage.json'):
    os.remove('/tmp/repl-storage.json')

# So that the all-clusters-app won't boot with stale prior state.    
os.system('rm -rf /tmp/chip_*')

## Initialization

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

`ChipReplStartup.py` is run within the global namespace. This results in all of its imports being made available here.

> **NOTE**: _This is not needed if you launch the REPL from the command-line._

In [2]:
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path}

──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

2022-01-25 16:58:57 johnsj-macbookpro1.roam.corp.google.com root[27921] ERROR [Errno 2] No such file or directory: '/tmp/repl-storage.json'
2022-01-25 16:58:57 johnsj-macbookpro1.roam.corp.google.com root[27921] WARNING Could not load configuration from /tmp/repl-storage.json - resetting configuration...


No previous fabric admins discovered in persistent storage - creating a new one...

New FabricAdmin: FabricId: 1(1)


Creating default device controller on fabric 1...

Allocating new controller with FabricId: 1(1), NodeId: 1


Default CHIP Device Controller has been initialized to manage fabricAdmins[0], and is 
available as devCtrl

At startup, the REPL will attempt to find any previously configured fabrics stored in persisted storage. If it can't find any (as is the case here), it will construct a default `FabricAdmin` object on Fabric 1 (Index 1) as well as construct a device controller (`devCtrl`) on that fabric.

In [3]:
fabricAdmins

[
│   <chip.FabricAdmin.FabricAdmin object at 0x7feacf3d1dc0>
]

You can check the underlying fabric info by typing the name directly in the repl.

In [4]:
devCtrl

<Controller for Fabric 0000000000000001 (Compressed Fabric Id: b75777b5840b9309)>

### Commission onto Fabric 1

#### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [ ]:
import time, os
import subprocess
os.system('pkill -f chip-all-clusters-app')
time.sleep(1)

# The location of the all-clusters-app in the cloud playground is one level higher - adjust for this by testing for file presence.
if (os.path.isfile('../../../out/debug/chip-all-clusters-app')):
    appPath = '../../../out/debug/chip-all-clusters-app'
else:
    appPath = '../../../../out/debug/chip-all-clusters-app'

process = subprocess.Popen(appPath, stdout=subprocess.DEVNULL)
time.sleep(1)

#### Commission Target

Commission the target onto Fabric 1 using the default device controller instance with a NodeId of 1.

In [6]:
devCtrl.CommissionIP(b'127.0.0.1', 20202021, 2)

2022-03-21 17:50:16 songguo.sha.corp.google.com chip.CTL[2635279] ERROR Unable to find country code, defaulting to XX
2022-03-21 17:50:16 songguo.sha.corp.google.com chip.DL[2635279] ERROR Avahi resolve found
2022-03-21 17:50:16 songguo.sha.corp.google.com chip.SC[2635279] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


Commissioning complete


True

### Read OpCreds Cluster

Read out the OpCreds cluster to confirm membership into Fabric 1.

In [8]:
await devCtrl.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.Fabrics)], fabricFiltered=False)

{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 2299082310,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.Fabrics'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   rootPublicKey=b'\x04\xba\x07f\xa8`\xfeh\xefe\x9eh\n\xda\x93\xb4nc\xdb\xcb\x89\xc2x]I\xaf#\x8c\xbc\xa7~5\x14$\xd7h\x84\x03\xf0-,A\x06\xcd\xa0\xc8\t\xca\xa7!\x06\xefc!B`3\xe9H\xfa\xf9\x97):\x11',
│   │   │   │   │   vendorId=28446,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='',
│   │   │   │   │   fabricIndex=1
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

### Commission onto Fabric 2

#### Create new FabricAdmin

In [9]:
import chip.FabricAdmin as FabricAdmin
fabric2 = FabricAdmin.FabricAdmin(fabricId = 2, fabricIndex = 2)

New FabricAdmin: FabricId: 2(2)


Here's a brief peek at the JSON data that is in the persisted storage file.

In [10]:
builtins.chipStack.GetStorageManager().jsonData

{
│   'repl-config': {
│   │   'fabricAdmins': {
│   │   │   '1': {
│   │   │   │   'fabricId': 1
│   │   │   },
│   │   │   '2': {
│   │   │   │   'fabricId': 2
│   │   │   }
│   │   }
│   },
│   'sdk-config': {
│   │   'gcc': '6AMAAA==',
│   │   'gdc': '6AMAAA==',
│   │   'ExampleOpCredsCAKey1': 'BLoHZqhg/mjvZZ5oCtqTtG5j28uJwnhdSa8jjLynfjUUJNdohAPwLSxBBs2gyAnKpyEG72MhQmAz6Uj6+ZcpOhEiwYZIs1Zs8jTWTwIfrnpBbKivp6iocOqEF14G0KLoOw==',
│   │   'ExampleOpCredsICAKey1': 'BJDYC+BfoNm9VtxmUwt2UdGbsJvjF6NUMSTW/gdXMr9YwlGNd2WaJ5vTGb9pS4tWPOsCjqsRmhtLVuAWe68jniOjjr64anP1DcFQiSxOsrWmwlghxN4ps6oRdh1+wLIZ/A==',
│   │   'ExampleCAIntermediateCert1': 'MIIBlTCCATygAwIBAgIBADAKBggqhkjOPQQDAjAiMSAwHgYKKwYBBAGConwBBAwQMDAwMDAwMDAwMDAwMDAwMDAeFw0yMTAxMDEwMDAwMDBaFw0zMDEyMzAwMDAwMDBaMCIxIDAeBgorBgEEAYKifAEDDBAwMDAwMDAwMDAwMDAwMDAxMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEkNgL4F+g2b1W3GZTC3ZR0Zuwm+MXo1QxJNb+B1cyv1jCUY13ZZonm9MZv2lLi1Y86wKOqxGaG0tW4BZ7ryOeI6NjMGEwDwYDVR0TAQH/BAUwAwEB/zAOBgNVHQ8BAf8EBAMCAQYwHQYDVR0OBBYEFPx8CeluC/tiLeUZyboeJHaibmnZMB8GA1UdIwQYMBaAFHV8XorSBpWQpPrTWaniIKzUMQQUMAoGCCqGSM49BAMCA0cAMEQCIDZarzIwBowTlNEEYOG+W2qAiv0+7a8WZrV+4Oj8imfwAiB7uMT2Cu2grqAychG6QGyn9pirJb1ITc6rohp3kcJzZQ==',
│   │   'ExampleCARootCert1': 'MIIBljCCATygAwIBAgIBADAKBggqhkjOPQQDAjAiMSAwHgYKKwYBBAGConwBBAwQMDAwMDAwMDAwMDAwMDAwMDAeFw0yMTAxMDEwMDAwMDBaFw0zMDEyMzAwMDAwMDBaMCIxIDAeBgorBgEEAYKifAEEDBAwMDAwMDAwMDAwMDAwMDAwMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEugdmqGD+aO9lnmgK2pO0bmPby4nCeF1JryOMvKd+NRQk12iEA/AtLEEGzaDICcqnIQbvYyFCYDPpSPr5lyk6EaNjMGEwDwYDVR0TAQH/BAUwAwEB/zAOBgNVHQ8BAf8EBAMCAQYwHQYDVR0OBBYEFHV8XorSBpWQpPrTWaniIKzUMQQUMB8GA1UdIwQYMBaAFHV8XorSBpWQpPrTWaniIKzUMQQUMAoGCCqGSM49BAMCA0gAMEUCIQDOKet8M81frd8SE9jEKGBoG8fP9EyELJU8rcr7imjFbQIgPPYKXr+qYmLu0UgdwG1gYA4DgwtroeBq08lXLI4vmiA=',
│   │   'f/1/r': 'FTABAQAkAgE3AyQUABgmBIAigScmBYAlTTo3BiQUABgkBwEkCAEwCUEEugdmqGD+aO9lnmgK2pO0bmPby4nCeF1JryOMvKd+NRQk12iEA/AtLEEGzaDICcqnIQbvYyFCYDPpSPr5lyk6ETcKNQEpARgkAmAwBBR1fF6K0gaVkKT601mp4iCs1DEEFDAFFHV8XorSBpWQpPrTWaniIKzUMQQUGDALQM4p63wzzV+t3xIT2MQoYGgbx8/0TIQslTytyvuKaMVtPPYKXr+qYmLu0UgdwG1gYA4DgwtroeBq08lXLI4vmiAY',
│   │   'f/1/i': 'FTABAQAkAgE3AyQUABgmBIAigScmBYAlTTo3BiQTARgkBwEkCAEwCUEEkNgL4F+g2b1W3GZTC3ZR0Zuwm+MXo1QxJNb+B1cyv1jCUY13ZZonm9MZv2lLi1Y86wKOqxGaG0tW4BZ7ryOeIzcKNQEpARgkAmAwBBT8fAnpbgv7Yi3lGcm6HiR2om5p2TAFFHV8XorSBpWQpPrTWaniIKzUMQQUGDALQDZarzIwBowTlNEEYOG+W2qAiv0+7a8WZrV+4Oj8imfwe7jE9grtoK6gMnIRukBsp/aYqyW9SE3Oq6Iad5HCc2UY',
│   │   'f/1/n': 'FTABAQEkAgE3AyQTARgmBIAigScmBYAlTTo3BiQVASQRARgkBwEkCAEwCUEE7H1GfRAQAFIDbiSZ1dtU0Il6MbDWj7BAsn7x7ipBGMTremwb6wHJTgdYOn3TiLQyemtGpVPHSSrT/U0Eu/zXTDcKNQEoARgkAgE2AwQCBAEYMAQUlsEJfINpYHDYm44rs12a69sOI5EwBRT8fAnpbgv7Yi3lGcm6HiR2om5p2RgwC0Cxs6rv3m/CSr/Vvl6nqpjP+/UZlldwCzyTStX0bzxnLduY7WZMo8We5zSzW3y6/DwQ1jgzNKu+hBti1BeHZYCyGA==',
│   │   'f/1/o': 'FSQAATABYQTsfUZ9EBAAUgNuJJnV21TQiXoxsNaPsECyfvHuKkEYxOt6bBvrAclOB1g6fdOItDJ6a0alU8dJKtP9TQS7/NdMMlYH6llXytrjsG6QE8BgxWH1eA1jFzzVpFQVwjbhH7kY',
│   │   'f/1/m': 'FSUAHm8sAQAY',
│   │   'ExampleOpCredsCAKey2': 'BFXPy14Sye6FHxUEBSC3ZyT5A6V9JA376ysIqynHsvlNhcoVwau6SW+orHgQid3/STqlOx4VbHjtINkkYNyrKlt7VJaWA6n3jSUbGQPIepAyZMy01tRmLtLcuO+r+F9UPw==',
│   │   'ExampleOpCredsICAKey2': 'BMyJDqEcQpKfxOGH3RHaaIPJNTLjAhpvpq2HhCR4Y9tsdx0P6HhsCa5IHKE1qwZ8jgMrVLX+/Jv7vPGQOKDd+OvEs4PtAus9Z7wPohnLLKIjfTrLssJIr+cKR3/dQ24fEg=='
│   }
}

In [12]:
devCtrl2 = fabric2.NewController()
devCtrl2

2022-03-21 17:51:28 songguo.sha.corp.google.com chip.DL[2635279] ERROR Cannot set hostname on this system, continue anyway...


Allocating new controller with FabricId: 2(2), NodeId: 2


<Controller for Fabric 0000000000000002 (Compressed Fabric Id: b68c0efe11caa83c)>

#### Open Commissioning Window

In [14]:
await devCtrl.SendCommand(2, 0, Clusters.AdministratorCommissioning.Commands.OpenBasicCommissioningWindow(100), timedRequestTimeoutMs=1000)

In [15]:
devCtrl2.CommissionIP(b'127.0.0.1', 20202021, 2)

2022-01-25 16:59:00 johnsj-macbookpro1.roam.corp.google.com chip.CTL[27921] ERROR Unable to find country code, defaulting to WW
2022-01-25 16:59:00 johnsj-macbookpro1.roam.corp.google.com chip.SC[27921] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


Commissioning complete


True

### Read OpCreds Cluster

Read out the OpCreds cluster to confirm membership into Fabric 2.

In [16]:
await devCtrl2.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.Fabrics)], fabricFiltered=False)

{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 2299082327,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.Fabrics'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   rootPublicKey=b'\x04\xba\x07f\xa8`\xfeh\xefe\x9eh\n\xda\x93\xb4nc\xdb\xcb\x89\xc2x]I\xaf#\x8c\xbc\xa7~5\x14$\xd7h\x84\x03\xf0-,A\x06\xcd\xa0\xc8\t\xca\xa7!\x06\xefc!B`3\xe9H\xfa\xf9\x97):\x11',
│   │   │   │   │   vendorId=28446,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='',
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   FabricDescriptor(
│   │   │   │   │   rootPublicKey=b'\x04U\xcf\xcb^\x12\xc9\xee\x85\x1f\x15\x04\x05 \xb7g$\xf9\x03\xa5}$\r\xfb\xeb+\x08\xab)\xc7\xb2\xf9M\x85\xca\x15\xc1\xab\xbaIo\xa8\xacx\x10\x89\xdd\xffI:\xa5;\x1e\x15lx\xed \xd9$`\xdc\xab*[',
│   │   │   │   │   vendorId=56886,
│   │   │   │   │   fabricId=2,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='',
│   │   │   │   │   fabricIndex=2
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

## Relaunch REPL

Let's simulate re-launching the REPL to show-case the capabilities of the persistence storage and its mechanics.

In [17]:
import chip.native
import pkgutil
module = pkgutil.get_loader('chip.ChipReplStartup')
%run {module.path}

──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

Restoring FabricAdmin from storage to manage FabricId 1, FabricIndex 1...

New FabricAdmin: FabricId: 1(1)


...

New FabricAdmin: FabricId: 2(2)


Fabric Admins have been loaded and are available at fabricAdmins

Creating default device controller on fabric 1...

Allocating new controller with FabricId: 1(1), NodeId: 1


Default CHIP Device Controller has been initialized to manage fabricAdmins[0], and is 
available as devCtrl

The REPL has now loaded the two fabrics that were created in the previous session into the `fabricAdmins` variable. It has also created a default controller on the first fabric in that list (Fabric 1) as `devCtrl`.

### Establish CASE and Read OpCreds

To prove that we do indeed have two distinct fabrics and controllers on each fabric, let's go ahead and update the label of each fabric. To do so, you'd need to succcessfully establish a CASE session through a controller on the respective fabric, and call the 'UpdateLabel' command.

Underneath the covers, each device controller will do operational discovery of the NodeId being read and establish a CASE session before issuing the IM interaction.

In [19]:
await devCtrl.SendCommand(2, 0, Clusters.OperationalCredentials.Commands.UpdateFabricLabel("Fabric1Label"))
await devCtrl.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.Fabrics)], fabricFiltered=False)

{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 2299082347,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.Fabrics'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   rootPublicKey=b'\x04\xba\x07f\xa8`\xfeh\xefe\x9eh\n\xda\x93\xb4nc\xdb\xcb\x89\xc2x]I\xaf#\x8c\xbc\xa7~5\x14$\xd7h\x84\x03\xf0-,A\x06\xcd\xa0\xc8\t\xca\xa7!\x06\xefc!B`3\xe9H\xfa\xf9\x97):\x11',
│   │   │   │   │   vendorId=28446,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='Fabric1Label',
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   FabricDescriptor(
│   │   │   │   │   rootPublicKey=b'\x04U\xcf\xcb^\x12\xc9\xee\x85\x1f\x15\x04\x05 \xb7g$\xf9\x03\xa5}$\r\xfb\xeb+\x08\xab)\xc7\xb2\xf9M\x85\xca\x15\xc1\xab\xbaIo\xa8\xacx\x10\x89\xdd\xffI:\xa5;\x1e\x15lx\xed \xd9$`\xdc\xab*[',
│   │   │   │   │   vendorId=56886,
│   │   │   │   │   fabricId=2,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='',
│   │   │   │   │   fabricIndex=2
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

Instantiate a controller on fabric 2 and use it to read out the op creds from that fabric.

In [22]:
devCtrl2 = fabricAdmins[1].NewController()
await devCtrl2.SendCommand(2, 0, Clusters.OperationalCredentials.Commands.UpdateFabricLabel("Fabric2Label"))
await devCtrl2.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.Fabrics)], fabricFiltered=False)

2022-03-21 17:53:10 songguo.sha.corp.google.com chip.DL[2635279] ERROR Cannot set hostname on this system, continue anyway...
2022-03-21 17:53:10 songguo.sha.corp.google.com chip.DL[2635279] ERROR Avahi resolve found


Allocating new controller with FabricId: 2(2), NodeId: 2


2022-03-21 17:53:10 songguo.sha.corp.google.com chip.SC[2635279] ERROR The device does not support GetClock_RealTimeMS() API. This will eventually result in CASE session setup failures.


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 2299082367,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.Fabrics'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   rootPublicKey=b'\x04\xba\x07f\xa8`\xfeh\xefe\x9eh\n\xda\x93\xb4nc\xdb\xcb\x89\xc2x]I\xaf#\x8c\xbc\xa7~5\x14$\xd7h\x84\x03\xf0-,A\x06\xcd\xa0\xc8\t\xca\xa7!\x06\xefc!B`3\xe9H\xfa\xf9\x97):\x11',
│   │   │   │   │   vendorId=28446,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='Fabric1Label',
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   FabricDescriptor(
│   │   │   │   │   rootPublicKey=b'\x04U\xcf\xcb^\x12\xc9\xee\x85\x1f\x15\x04\x05 \xb7g$\xf9\x03\xa5}$\r\xfb\xeb+\x08\xab)\xc7\xb2\xf9M\x85\xca\x15\xc1\xab\xbaIo\xa8\xacx\x10\x89\xdd\xffI:\xa5;\x1e\x15lx\xed \xd9$`\xdc\xab*[',
│   │   │   │   │   vendorId=56886,
│   │   │   │   │   fabricId=2,
│   │   │   │   │   nodeId=2,
│   │   │   │   │   label='Fabric2Label',
│   │   │   │   │   fabricIndex=2
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

In [21]:
devCtrl2.Shutdown()